train

In [ ]:
!pip install git+https://github.com/darecophoenixx/wordroid.sblo.jp

In [ ]:
%matplotlib inline
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

import os.path
import sys
import re
import itertools
import csv
import datetime
import pickle
import random
from collections import defaultdict, Counter
import gc

import matplotlib.pyplot as plt
from matplotlib.ticker import NullFormatter
import seaborn as sns
import pandas as pd
import numpy as np
import scipy
import gensim
from sklearn.metrics import f1_score, classification_report, confusion_matrix, log_loss
from sklearn.model_selection import train_test_split
import gensim
from keras.preprocessing.sequence import skipgrams
import tensorflow as tf

In [ ]:
def hexbin(x, y, color, **kwargs):
    cmap = sns.light_palette(color, as_cmap=True)
    plt.hexbin(x, y, cmap=cmap, **kwargs)
def scatter(x, y, color, **kwargs):
    plt.scatter(x, y, marker='.')

# Prepare data

In [ ]:
ls -la ../input

In [ ]:
dir_data_src = '../input/anime-recommendations-database'
os.listdir(dir_data_src)

In [ ]:
dir_wd2v_p_src = '../input/anime-recommendations-wd2v-preparation-no-genre'
os.listdir(dir_wd2v_p_src)

In [ ]:
anime_csv = pd.read_csv(os.path.join(dir_data_src, 'anime.csv'))
print(anime_csv.shape)
anime_csv.index = anime_csv.anime_id.values
anime_csv.head()

In [ ]:
genre = []
for ee in [str(ee).replace(' ', '').split(',') for ee in anime_csv.genre.values]:
    genre.extend(ee)
genre = [str(ee) for ee in np.unique(genre)]

In [ ]:
anime_csv_dic = {}
for ii in range(anime_csv.shape[0]):
    anime_id = anime_csv.anime_id.values[ii]
    anime_name = anime_csv.name.values[ii]
    anime_csv_dic[anime_id] = anime_name

anime_csv_dic

In [ ]:
rating_csv = pd.read_csv(os.path.join(dir_data_src, 'rating.csv'))
print(rating_csv.shape)
rating_csv.head()

In [ ]:
rating_csv2 = rating_csv.loc[rating_csv.rating.values != -1]
rating_csv2.shape
rating_csv2.head(100)

In [ ]:
user_dic = {}
for ee in np.unique(rating_csv.user_id.values):
    user_dic[ee] = 'user_id-' + str(ee)

len(user_dic)

In [ ]:
from feature_eng import WordAndDoc2vec, calc_gsim

In [ ]:
src_dir = dir_wd2v_p_src
corpus_csr = scipy.sparse.load_npz(os.path.join(src_dir, 'corpus_csr.npz'))
dic_user = gensim.corpora.dictionary.Dictionary.load(os.path.join(src_dir, 'word_dic'))
dic_anime = gensim.corpora.dictionary.Dictionary.load(os.path.join(src_dir, 'doc_dic'))

In [ ]:
%%time
wd2v = WordAndDoc2vec(corpus_csr, dic_user, dic_anime)
wd2v

In [ ]:
len(wd2v.doc_dic), len(wd2v.word_dic)

# Train

In [ ]:
num_features = 128
wd2v.make_model(
    num_features=num_features, max_num_prod=50,
    maxnorm=1000, embeddings_val=0.1
)

In [ ]:
wgt_prod = wd2v.wgt_col
print(wgt_prod.shape)
df = pd.DataFrame(wgt_prod[:,:5])
sns.pairplot(df, markers='.')

In [ ]:
wgt_user = wd2v.wgt_row
print(wgt_user.shape)
df = pd.DataFrame(wgt_user[:,:5])
sns.pairplot(df, markers='.')

In [ ]:
%%time

from keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau

epoch0 = 16
def lr_schedule(epoch):
    lr0 = 0.02
    epoch1 = epoch0
    epoch2 = epoch0
    epoch3 = epoch0
    epoch4 = epoch0
    
    if epoch<epoch1:
        lr = lr0
    elif epoch<epoch1+epoch2:
        lr = lr0/2
    elif epoch<epoch1+epoch2+epoch3:
        lr = lr0/4
    elif epoch<epoch1+epoch2+epoch3+epoch4:
        lr = lr0/8
    else:
        lr = lr0/16
    
    if divmod(epoch,4)[1] == 3:
        lr *= (1/8)
    elif divmod(epoch,4)[1] == 2:
        lr *= (1/4)
    elif divmod(epoch,4)[1] == 1:
        lr *= (1/2)
    elif divmod(epoch,4)[1] == 0:
        pass
    print('Learning rate: ', lr)
    return lr

lr_scheduler = LearningRateScheduler(lr_schedule)

callbacks = [lr_scheduler]

hst = wd2v.train(epochs=epoch0*4, verbose=2,
           use_multiprocessing=True, workers=4,
           callbacks=callbacks)

In [ ]:
hst_history = hst.history

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(20,5))
ax[0].set_title('loss')
ax[0].plot(list(range(len(hst_history["loss"]))), hst_history["loss"], label="Train loss")
ax[1].set_title('binary_accuracy')
ax[1].plot(list(range(len(hst_history["loss"]))), hst_history["binary_accuracy"], label="binary_accuracy")
ax[2].set_title('learning rate')
ax[2].plot(list(range(len(hst_history["loss"]))), hst_history["lr"], label="learning rate")
ax[0].legend()
ax[1].legend()
ax[2].legend()

In [ ]:
np.save('wgt_col', wd2v.wgt_col)
np.save('wgt_row', wd2v.wgt_row)

In [ ]:
[dic_user[k] for k in dic_user.keys()]

In [ ]:
df_wgt_col = pd.DataFrame(wd2v.wgt_col, index=[dic_user[k] for k in dic_user.keys()])
print(df_wgt_col.shape)
df_wgt_col

In [ ]:
df_wgt_row = pd.DataFrame(wd2v.wgt_row, index=[dic_anime[k] for k in dic_anime.keys()])
print(df_wgt_row.shape)
df_wgt_row

In [ ]:
df_wgt_col.to_csv('wgt_col.csv')
df_wgt_row.to_csv('wgt_row.csv')

In [ ]:
os.listdir('.')

In [ ]:
wgt_prod = wd2v.wgt_col
print(np.sqrt(np.square(wgt_prod).sum(axis=1)))
print(wgt_prod.shape)
df = pd.DataFrame(wgt_prod[:,:5])
#df['cls'] = ['ph'] + ['c'+str(ii) for ii in cls_prod]
df.head()

In [ ]:
sns.pairplot(df, markers='.', height=3.5, diag_kind='hist')

In [ ]:
wgt_user = wd2v.wgt_row
print(np.sqrt(np.square(wgt_user).sum(axis=1)))
print(wgt_user.shape)
df = pd.DataFrame(wgt_user[:,:5])
#df['cls'] = ['c'+str(ii) for ii in cls_user]
df.head()

In [ ]:
sns.pairplot(df, markers='.', height=3.5, diag_kind='hist')

# Get Similarity

In [ ]:
sim = wd2v.sim
sim

In [ ]:
sim.row_dic

In [ ]:
anime_csv_dic[32281]

In [ ]:
query = sim.get_fet_byrow('anime_id-100')
query

In [ ]:
sim.get_sim_bycol(query)

In [ ]:
sim.get_sim_byrow(query)